In [1]:
import json
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.ticker import ScalarFormatter, FormatStrFormatter,StrMethodFormatter
from matplotlib.ticker import FormatStrFormatter
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
import warnings
from pandas.core.common import SettingWithCopyWarning

In [2]:
portfolio = pd.read_json(r'C:\Desktop\portfolio.json', orient='records', lines=True)
profile = pd.read_json(r'C:\Desktop\profile.json', orient='records', lines=True)
transcript = pd.read_json(r'C:\Desktop\transcript.json', orient='records', lines=True)

In [3]:
df=pd.DataFrame(transcript['value'].values.tolist(),index=transcript.index)

In [4]:
transcript=transcript.merge(df,left_index=True, right_index=True)

In [5]:
#one hot encode channels
mlb = MultiLabelBinarizer()
channel = pd.DataFrame(mlb.fit_transform(portfolio['channels']),columns=mlb.classes_, index=portfolio.index)

In [6]:
profile.head()

,gender,age,id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
4,None,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN


In [7]:
#there are 17000 users
transcript['person'].nunique()

17000

In [8]:
#add column called week
#event=transaction has an amount.
order_events = transcript[transcript['event'].isin(['transaction'])]
pd.set_option('display.max_columns', None)
order_events = order_events.copy()
order_events['week'] = (order_events['time'] // (24 * 7)) + 1
order_events['day'] = order_events['time'] //  (24) + 1 

In [9]:
order_events_day1to15 = order_events[order_events['day'].between(1, 15)]

order_events_day1to15['day'].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15],
      dtype=int64)

In [10]:
# Group by person and find the maximum day
max_day_by_user = order_events_day1to15.groupby('person')['day'].max().reset_index()

day_15=15
# Calculate recency
max_day_by_user['recency'] = day_15 - max_day_by_user['day']

# Merge with the order_events_day1to15 dataframe
order_events_day1to15 = pd.merge(order_events_day1to15, max_day_by_user[['person', 'recency']], on='person')

In [11]:
order_events_day1to15['recency'].unique()

array([ 6,  0,  1,  5, 14,  4, 10,  2,  3,  7, 12,  9,  8, 11, 13],
      dtype=int64)

In [12]:
#filter users who made a purchase between day 1-15 greater than $0.
order_events_day1to15 = order_events_day1to15.groupby('person').\
agg({'recency': 'first', 'event': 'size','amount': lambda x: x[x > 0].sum()\
    }).reset_index()
# Rename columns
order_events_day1to15.rename(columns={
    'recency': 'Recency (# Days ago from last trans. As of final day up to Day 15)',
    'event': 'Frequency (# Trans. from Day 1-15)',
    'amount': 'Monetary Value (Sum of Trans. from Day 1-15)',
}, inplace=True)

In [13]:
unique_person_count = order_events_day1to15['person'].nunique()
print(unique_person_count)

15042


In [14]:
order_events_day1to15=order_events_day1to15.merge(profile, left_on='person', right_on='id', how='left')

In [15]:
order_events_day1to15.drop('id', axis=1, inplace=True)

In [16]:
# Convert 'became_member_on' column to datetime format
order_events_day1to15['became_member_on'] = pd.to_datetime(order_events_day1to15['became_member_on'], format='%Y%m%d')

# Extract year from the 'became_member_on' column
order_events_day1to15['member_year'] = order_events_day1to15['became_member_on'].dt.year
order_events_day1to15=order_events_day1to15.drop('became_member_on', axis=1)

In [17]:
order_events_day1to15['gender'].fillna('Unknown', inplace=True)
order_events_day1to15['income'].fillna('Unknown', inplace=True)
order_events_day1to15['age'].replace(118, 'Unknown', inplace=True)

In [18]:
bins = [18, 35, 51, 68, 85, 102]
labels = ['18-34', '35-50', '51-67', '68-84', '85-101']

# Convert 'age' column to numeric
order_events_day1to15['age'] = pd.to_numeric(order_events_day1to15['age'], errors='coerce')

# Bin the 'age' column
order_events_day1to15['age_group'] = pd.cut(order_events_day1to15['age'], bins=bins, labels=labels, right=False)

In [19]:
bins= [30000,50001,75001,100001,120001]
labels = ['30k-50k','50k-75k','75k-100k','100k-120k']
# Convert column to numeric
order_events_day1to15['income'] = pd.to_numeric(order_events_day1to15['income'], errors='coerce')

order_events_day1to15['income_group']=pd.cut(order_events_day1to15['income'], bins=bins, labels=labels,right=False)

In [20]:
order_events_day1to15=order_events_day1to15.drop(['age','income'], axis=1)

In [21]:
#Create AOV column
order_events_day1to15['Avg. Order Value (AOV) from Day 1-15 (Monetary Value/Frequency)'] = \
    round(order_events_day1to15['Monetary Value (Sum of Trans. from Day 1-15)'] / \
          order_events_day1to15['Frequency (# Trans. from Day 1-15)'],2)

In [22]:
order_events_day1to15 = order_events_day1to15[['person','age_group', 'gender', 'income_group', 'member_year', 
                                               'Recency (# Days ago from last trans. As of final day up to Day 15)',
                                               'Frequency (# Trans. from Day 1-15)',
                                               'Monetary Value (Sum of Trans. from Day 1-15)',
                                               'Avg. Order Value (AOV) from Day 1-15 (Monetary Value/Frequency)']]

In [23]:
order_events_day1to15=order_events_day1to15.rename(columns={'person':'Customer ID',
    'age_group': 'Age', 'gender': 'Gender', 'income_group': 'HH Income', 'member_year': 'MemberSince'})

In [24]:
order_events_day16to30 = order_events[order_events['day'].between(16, 30)]
order_events_day16to30['day'].unique()

array([16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30],
      dtype=int64)

In [25]:
order_events_day16to30 = order_events_day16to30.groupby('person').\
agg({'amount': lambda x: x[x > 0].sum()}).reset_index()
order_events_day16to30.head()

,person,amount
0,0009655768c64bdeb2e877511632db8f,105.44
1,00116118485d4dfda04fdbaba9a87b5c,3.39
2,0011e0d4e6b944f998e987f904e8c1e5,54.04
3,0020c2b971eb4e9188eac86d93036a77,98.53
4,0020ccbbb6d84e358d3414a3ff76cffd,91.19


In [26]:
sbux_clv = order_events_day1to15.merge(order_events_day16to30, left_on='Customer ID', right_on='person', how='left')

In [27]:
sbux_clv['Age'] = sbux_clv['Age'].cat.add_categories('Unknown').fillna('Unknown')
sbux_clv['HH Income'] = sbux_clv['HH Income'].cat.add_categories('Unknown').fillna('Unknown')
sbux_clv = sbux_clv.drop('person', axis=1)

In [28]:
sbux_clv=sbux_clv.rename(columns={'amount': 'Sum(Trans. Amt from Day 16-30)'})

In [29]:
#event=offers_received
order_events = transcript[transcript['event'].isin(['offer received'])]
pd.set_option('display.max_columns', None)
order_events = order_events.copy()
order_events['week'] = (order_events['time'] // (24 * 7)) + 1
order_events['day'] = order_events['time'] //  (24) + 1 

In [30]:
order_events_day1to15 = order_events[order_events['day'].between(1, 15)]

order_events_day1to15['day'].unique()

array([ 1,  8, 15], dtype=int64)

In [31]:
#count number of offer received by person from Day 1-15
offer_received_counts = order_events_day1to15[order_events_day1to15['event'] == 'offer received']['person'].value_counts().to_frame().reset_index()
offer_received_counts.columns = ['person', 'offer_received_count']

In [32]:
#event=offer completed
order_events = transcript[transcript['event'].isin(['offer completed'])]
pd.set_option('display.max_columns', None)
order_events = order_events.copy()
order_events['week'] = (order_events['time'] // (24 * 7)) + 1
order_events['day'] = order_events['time'] //  (24) + 1 

In [33]:
order_events_day1to15 = order_events[order_events['day'].between(1, 15)]
order_events_day1to15['day'].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15],
      dtype=int64)

In [34]:
#count number of offer completed by person from Day 1-15
offer_completed_count = order_events_day1to15[order_events_day1to15['event'] == 'offer completed']['person'].value_counts().to_frame().reset_index()
offer_completed_count.columns = ['person', 'offer_completed_count']

In [35]:
#event=offer viewed
order_events = transcript[transcript['event'].isin(['offer viewed'])]
pd.set_option('display.max_columns', None)
order_events = order_events.copy()
order_events['week'] = (order_events['time'] // (24 * 7)) + 1
order_events['day'] = order_events['time'] //  (24) + 1 

In [36]:
order_events_day1to15 = order_events[order_events['day'].between(1, 15)]
order_events_day1to15['day'].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15],
      dtype=int64)

In [37]:
#count number of offer viewed by person from Day 1-15
offer_viewed_count = order_events_day1to15[order_events_day1to15['event'] == 'offer viewed']['person'].value_counts().to_frame().reset_index()
offer_viewed_count.columns = ['person', 'offer_viewed_count']

In [38]:
# Left join with offer_received_counts
sbux_clv_with_received = pd.merge(sbux_clv, offer_received_counts, left_on='Customer ID', right_on='person', how='left')

In [39]:
#check
sbux_clv_with_received = sbux_clv_with_received.drop(['person'], axis=1)

In [40]:
# Left join with offer_completed_count
sbux_clv_with_received_completed = pd.merge(sbux_clv_with_received, offer_completed_count, left_on='Customer ID', 
                                            right_on='person', how='left')

In [41]:
#check
sbux_clv_with_received_completed = sbux_clv_with_received_completed.drop(['person'], axis=1)

In [42]:
sbux_clv_with_received_completed['Marketing Offer Response Rate from Day 1-15'] = \
round(sbux_clv_with_received_completed['offer_completed_count'].fillna(0) / sbux_clv_with_received_completed['offer_received_count'].replace(0, float('nan')),2)

In [43]:
# Left join with offer_viewed_count
sbux_clv_with_viewed_completed = pd.merge(sbux_clv_with_received_completed, offer_viewed_count, left_on='Customer ID', 
                                            right_on='person', how='left')

sbux_clv_with_viewed_completed['Marketing Offer View Rate from Day 1-15'] = \
round(sbux_clv_with_viewed_completed['offer_viewed_count'].fillna(0) / sbux_clv_with_viewed_completed['offer_received_count'].replace(0, float('nan')),2)

#check
sbux_clv_with_viewed_completed = sbux_clv_with_viewed_completed.drop(['person'], axis=1)

In [44]:
sbux_clv_with_viewed_completed = sbux_clv_with_viewed_completed.rename(columns={
    'offer_received_count': '# of Marketing offers that Starbucks sent to each customer from Day 1-15',
    'offer_completed_count': '# of Marketing offers that were successfully completed from Day 1-15',
    'offer_viewed_count': '# of Marketing offers that were viewed from Day 1-15'
})
#not everyone received an offer in the first 15 days
#not everyone viewed an offer in the first 15 days
#not everyone completed an offer in the first 15 days

In [45]:
portfolio=portfolio.merge(channel,left_index=True, right_index=True)

In [46]:
#offer received by channel
transcript=transcript.merge(portfolio, left_on='offer_id', right_on='id', how='left')

In [47]:
order_events = transcript[transcript['event'].isin(['offer completed'])]
pd.set_option('display.max_columns', None)
order_events = order_events.copy()
order_events['week'] = (order_events['time'] // (24 * 7)) + 1
order_events['day'] = order_events['time'] //  (24) + 1 

In [48]:
order_events_day1to15 = order_events[order_events['day'].between(1, 15)]

order_events_day1to15['day'].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15],
      dtype=int64)

In [49]:
order_events_day1to15=order_events_day1to15.groupby('person')[['email', 'mobile', 'social', 'web']].sum().reset_index()

In [50]:
order_events_day1to15.rename(columns={'email': 'Count of Completed offers using Email channel from Day 1-15',
                   'mobile': 'Count of Completed offers using Mobile channel from Day 1-15',
                   'social': 'Count of Completed offers using Social channel from Day 1-15',
                   'web': 'Count of Completed offers using Web channel from Day 1-15'}, inplace=True)
sbux_clv_with_viewed_completed_channel = sbux_clv_with_viewed_completed.merge(order_events_day1to15, left_on='Customer ID', right_on='person', how='left')

In [51]:
sbux_clv_with_viewed_completed_channel=sbux_clv_with_viewed_completed_channel.drop('person', axis=1)

In [52]:
sbux_clv_with_viewed_completed_channel=sbux_clv_with_viewed_completed_channel[['Customer ID','Age','Gender','HH Income','MemberSince',
                                               'Recency (# Days ago from last trans. As of final day up to Day 15)',
                                               'Frequency (# Trans. from Day 1-15)',
                                               'Monetary Value (Sum of Trans. from Day 1-15)',
                                               'Avg. Order Value (AOV) from Day 1-15 (Monetary Value/Frequency)',
                                     '# of Marketing offers that Starbucks sent to each customer from Day 1-15',
                                     '# of Marketing offers that were viewed from Day 1-15',                                              
                                     '# of Marketing offers that were successfully completed from Day 1-15',
                                     'Marketing Offer View Rate from Day 1-15',
                                     'Marketing Offer Response Rate from Day 1-15',
                                     'Count of Completed offers using Email channel from Day 1-15',
                                     'Count of Completed offers using Mobile channel from Day 1-15',
                                     'Count of Completed offers using Social channel from Day 1-15',
                                     'Count of Completed offers using Web channel from Day 1-15',
                                     'Sum(Trans. Amt from Day 16-30)']]
sbux_clv_with_viewed_completed_channel

,Customer ID,Age,Gender,HH Income,MemberSince,Recency (# Days ago from last trans. As of final day up to Day 15),Frequency (# Trans. from Day 1-15),Monetary Value (Sum of Trans. from Day 1-15),Avg. Order Value (AOV) from Day 1-15 (Monetary Value/Frequency),# of Marketing offers that Starbucks sent to each customer from Day 1-15,# of Marketing offers that were viewed from Day 1-15,# of Marketing offers that were successfully completed from Day 1-15,Marketing Offer View Rate from Day 1-15,Marketing Offer Response Rate from Day 1-15,Count of Completed offers using Email channel from Day 1-15,Count of Completed offers using Mobile channel from Day 1-15,Count of Completed offers using Social channel from Day 1-15,Count of Completed offers using Web channel from Day 1-15,Sum(Trans. Amt from Day 16-30)
0,0009655768c64bdeb2e877511632db8f,18-34,M,50k-75k,2017,5,1,22.16,22.16,2.0,1.0,NaN,0.50,0.00,NaN,NaN,NaN,NaN,105.44
1,00116118485d4dfda04fdbaba9a87b5c,Unknown,Unknown,Unknown,2018,2,1,0.70,0.70,1.0,1.0,NaN,1.00,0.00,NaN,NaN,NaN,NaN,3.39
2,0011e0d4e6b944f998e987f904e8c1e5,35-50,O,50k-75k,2018,4,2,25.42,12.71,3.0,3.0,1.0,1.00,0.33,1.0,1.0,1.0,1.0,54.04
3,0020c2b971eb4e9188eac86d93036a77,51-67,F,75k-100k,2016,8,4,98.33,24.58,3.0,1.0,1.0,0.33,0.33,1.0,1.0,1.0,1.0,98.53
4,0020ccbbb6d84e358d3414a3ff76cffd,18-34,F,50k-75k,2016,2,5,62.86,12.57,2.0,2.0,1.0,1.00,0.50,1.0,1.0,1.0,1.0,91.19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15037,fff3ba4757bd42088c044ca26d73817a,68-84,F,75k-100k,2015,1,8,148.84,18.60,3.0,2.0,2.0,0.67,0.67,2.0,2.0,1.0,2.0,432.14
15038,fff7576017104bcc8677a8d63322b5e1,68-84,M,50k-75k,2017,3,3,16.05,5.35,2.0,2.0,1.0,1.00,0.50,1.0,1.0,1.0,1.0,13.89
15039,fff8957ea8b240a6b5e634b6ee8eafcf,68-84,M,50k-75k,2018,14,1,0.64,0.64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.51
15040,fffad4f4828548d1b5583907f2e9906b,18-34,M,30k-50k,2017,1,7,42.50,6.07,2.0,2.0,1.0,1.00,0.50,1.0,1.0,1.0,1.0,46.33


In [53]:
sbux_clv_with_viewed_completed_channel.to_csv(r'C:\Desktop\sbux_clv_with_viewed_completed_channel.csv', index=False)